In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u
import scipy.signal

c = const.c
G = const.G
AU = (1*u.au).to('m')
M_sun = const.M_sun
M_earth = const.M_earth

### Basic utility functions

In [ ]:
def rel_len(x_1, x_2):
    r_vec = x_1 - x_2
    r = np.linalg.norm(r_vec)
    return r

def findrCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = (m_1*x_1 + m_2*x_2)/M
    return r_cm

def γ(v, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    return 1/np.sqrt(1 - np.dot(v,v)/c**2)

### Acceleration equation and integration functions

In [ ]:
def accelerationN(r_vec, v_vec, p_vec, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    r = np.linalg.norm(r_vec)

    a = - G*(m_1 + m_2)*r_vec/r**3
    return a

def accelerationEIH(r_vec, v_vec, p_vec, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G

    r = np.linalg.norm(r_vec)
    v = np.linalg.norm(v_vec)
    
    r_dot = np.dot(r_vec, v_vec)/r

    # WHAT ABOUT γ?!!?!?!
    p = np.linalg.norm(p_vec)

    p_vec_dot = G**2*m_1*m_2*(m_1+m_2)/(c**2*r**4)*r_vec - G/r**3*( (m_1*m_2 + (1/2 + 3/2*(m_1 + m_2)**2/(m_1*m_2))*p**2/c**2 + 3/2*np.dot(p_vec, r_vec)**2/(c*r)**2)*r_vec - np.dot(p_vec, r_vec)/c**2*p_vec )

    # q_vec_dot = (1/m_1 + 1/m_2)*p_vec - 1/2*(1/m_1**3 + 1/m_2**3)/c**2*p**2*p_vec - G/r*( (m_1*m_2 + 3*(m_1 + m_2)**2/(m_1*m_2))/c**2*p_vec_dot + np.dot(p_vec, r_vec)/(c*r)**2*r_vec )

    a = (1/m_1 + 1/m_2)*p_vec_dot - 3/2*(1/m_1**3 + 1/m_2**3)/c**2*p**2*p_vec_dot - G/r*( -r_dot/r*((m_1*m_2 + 3*(m_1 + m_2)**2/(m_1*m_2))/c**2*p_vec + np.dot(p_vec, r_vec)/(c*r)**2*r_vec ) + (m_1*m_2 + 3*(m_1 + m_2)**2/(m_1*m_2))/c**2*p_vec_dot + ( (np.dot(p_vec_dot, r_vec) + np.dot(p_vec, v_vec))*r_vec + np.dot(p_vec, r_vec)*v_vec  )/(c*r)**2 - 2*np.dot(p_vec, r_vec)/(c**2*r**3)*r_vec*r_dot )

    return a


def acceleration1PM(r_vec, v_vec, p_vec, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    r = np.linalg.norm(r_vec)
    v = np.linalg.norm(v_vec)
    p = np.linalg.norm(p_vec)
    
    r_dot = np.dot(r_vec, v_vec)/r


    E_1 = np.sqrt((m_1*c**2)**2 + (p*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p*c)**2)
    c_1 = (m_1*m_2*c**2)**2 - 2*(E_1*E_2/c**2 + p**2)**2

    p_vec_dot = G*c_1/(E_1*E_2*r**3)*r_vec

    dE_1dp = p_vec*c**2/E_1
    dE_2dp = p_vec*c**2/E_2
    E_1_dot = np.dot(dE_1dp, p_vec_dot)
    E_2_dot = np.dot(dE_2dp, p_vec_dot)
    dE_1dp_dot = p_vec_dot*c**2/E_1 - p_vec*c**2/E_1**2*E_1_dot
    dE_2dp_dot = p_vec_dot*c**2/E_2 - p_vec*c**2/E_2**2*E_2_dot

    dc_1dp = -4*(E_1*E_2/c**2 + p**2)*((dE_1dp*E_2 + E_1*dE_2dp)/c**2 + 2*p_vec)
    c_1_dot = -4*(E_1*E_2/c**2 + p**2)*((E_1_dot*E_2 + E_1*E_2_dot)/c**2 + 2*np.dot(p_vec, p_vec_dot))
    dc_1dp_dot = -4*( ((E_1_dot*E_2 + E_1*E_2_dot)/c**2 + 2*np.dot(p_vec,p_vec_dot))*((dE_1dp*E_2 + E_1*dE_2dp)/c**2 + 2*p_vec) + (E_1*E_2/c**2 + p**2)*((dE_1dp_dot*E_2 + dE_1dp*E_2_dot + E_1_dot*dE_2dp + E_1*dE_2dp_dot)/c**2 + 2*p_vec_dot) )

    # q_dot = 

    a = dE_1dp_dot + dE_2dp_dot + G/(E_1**2*E_2**2*r)*(  -( r_dot/r + 2*E_1_dot/E_1 + 2*E_2_dot/E_2 )*( E_1*E_2*dc_1dp - c_1*(dE_1dp*E_2 + E_1*dE_2dp) ) + (E_1_dot*E_2 + E_1*E_2_dot)*dc_1dp + E_1*E_2*dc_1dp_dot - c_1_dot*(dE_1dp*E_2 + E_1*dE_2dp) - c_1*(dE_1dp_dot*E_2 + dE_1dp*E_2_dot + E_1_dot*dE_2dp + E_1*dE_2dp_dot) )

    return a

def acceleration2PM(r_vec, v_vec, p_vec, m_1, m_2, DL):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    r = np.linalg.norm(r_vec)
    v = np.linalg.norm(v_vec)
    p = np.linalg.norm(p_vec)
    
    r_dot = np.dot(r_vec, v_vec)/r


    E_1 = np.sqrt((m_1*c**2)**2 + (p*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p*c)**2)
    c_1 = (m_1*m_2*c**2)**2 - 2*(E_1*E_2/c**2 + p**2)**2
    c_r = 3*m_1**2*((m_1*m_2)**2*c**4 - 5*(E_1*E_2/c**2 + p**2)**2)
    c_l = (m_2/m_1)**2*c_r

    p_vec_dot = G*c_1/(E_1*E_2*r**3)*r_vec + G**2/r**4*2/(E_1*E_2)*r_vec*( 1/4*(c_r/m_1 + c_l/m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2) * (c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 + p**2)/c**2)  )
    p_dot = np.linalg.norm(p_vec_dot)

    dE_1dp = p_vec*c**2/E_1
    dE_2dp = p_vec*c**2/E_2
    E_1_dot = np.dot(dE_1dp, p_vec_dot)
    E_2_dot = np.dot(dE_2dp, p_vec_dot)
    dE_1dp_dot = p_vec_dot*c**2/E_1 - p_vec*c**2/E_1**2*E_1_dot
    dE_2dp_dot = p_vec_dot*c**2/E_2 - p_vec*c**2/E_2**2*E_2_dot

    green = E_1_dot*E_2 + E_1*E_2_dot
    purple = dE_1dp*E_2 + E_1*dE_2dp
    yellow = dE_1dp_dot*E_2 + dE_1dp*E_2_dot + E_1_dot*dE_2dp + E_1*dE_2dp_dot


    dc_1dp = -4*(E_1*E_2/c**2 + p**2)*((dE_1dp*E_2 + E_1*dE_2dp)/c**2 + 2*p_vec)
    c_1_dot = -4*(E_1*E_2/c**2 + p**2)*((E_1_dot*E_2 + E_1*E_2_dot)/c**2 + 2*np.dot(p_vec, p_vec_dot))
    dc_1dp_dot = -4*( ((E_1_dot*E_2 + E_1*E_2_dot)/c**2 + 2*np.dot(p_vec,p_vec_dot))*((dE_1dp*E_2 + E_1*dE_2dp)/c**2 + 2*p_vec) + (E_1*E_2/c**2 + p**2)*((dE_1dp_dot*E_2 + dE_1dp*E_2_dot + E_1_dot*dE_2dp + E_1*dE_2dp_dot)/c**2 + 2*p_vec_dot) )
    dc_rdp = -30*m_1**2*(E_1*E_2/c**2 + p**2)*(purple/c**2 + 2*p_vec)
    c_r_dot = -30*m_1**2*(E_1*E_2/c**2 + p**2)*(green/c**2 + 2*np.dot(p_vec, p_vec_dot))
    dc_rdp_dot = -30*m_1**2*( (green/c**2 + 2*np.dot(p_vec, p_vec_dot))*(purple/c**2 + 2*p_vec) + (E_1*E_2/c**2 + p**2)*(yellow/c**2 + 2*p_vec_dot) )

    # q_dot = dE_1dp + dE_2dp + G/(E_1**2*E_2**2*r)*( E_1*E_2*dc_1dp - c_1*(dE_1dp*E_2 + E_1*dE_2dp) )

    a_kin = dE_1dp_dot + dE_2dp_dot
    a_1PM = G/(E_1**2*E_2**2*r)*(  -( r_dot/r + 2*E_1_dot/E_1 + 2*E_2_dot/E_2 )*( E_1*E_2*dc_1dp - c_1*(dE_1dp*E_2 + E_1*dE_2dp) ) + (E_1_dot*E_2 + E_1*E_2_dot)*dc_1dp + E_1*E_2*dc_1dp_dot - c_1_dot*(dE_1dp*E_2 + E_1*dE_2dp) - c_1*(dE_1dp_dot*E_2 + dE_1dp*E_2_dot + E_1_dot*dE_2dp + E_1*dE_2dp_dot) )
    a_2PM = - G**2/(r**2*E_1*E_2)*(2*r_dot/r + green/E_1*E_2)  *  ( -purple/(E_1*E_2)*(1/4*(c_r/m_1 + c_l/m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2)*(c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 + p**2)/c**2 ) )  +  1/4*(1/m_1 + m_2/m_1**2)*1/c**2*dc_rdp + ( (E_1 + E_2)/(E_1*E_2)*dc_1dp + c_1/(E_1*E_2)*(dE_1dp + dE_2dp - (E_1 + E_2)/(E_1*E_2)*purple ) ) * (c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 + p**2)/c**2)  +  c_1*(E_1 + E_2)/(E_1*E_2) * ( 1/2*dc_1dp*( 1/(E_1 + E_2)**2 - 1/(E_1*E_2) ) + c_1/2*(-2*(dE_1dp + dE_2dp)/(E_1 + E_2)**3 + 1/(E_1*E_2)**2*purple) - 4/c**2*(purple/c**2 + 2*p_vec) ) )  +  G**2/(r**2*E_1*E_2) * ( -( yellow/(E_1*E_2) - purple/(E_1*E_2)**2*green ) * ( 1/4*(c_r/m_1 + c_l/m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2)*(c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 + p**2)/c**2 ) )  -  purple/(E_1*E_2) * ( 1/4*(1/m_1 + m_2/m_1**2)*c_r_dot/c**2 + ( (c_1_dot*(E_1 + E_2) + c_1*(E_1_dot + E_2_dot))/(E_1*E_2) - c_1*(E_1 + E_2)/(E_1*E_2)**2*green) * ( c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 +  p**2)/c**2 )  +  c_1*(E_1 + E_2)/(E_1*E_2) * ( c_1_dot/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) + c_1/2*(-2*(E_1_dot + E_2_dot)/(E_1 + E_2)**3 + green/(E_1*E_2)**2) - 4/c**2*(green/c**2 + 2*np.dot(p_vec, p_vec_dot) ) ) ) + 1/4*(1/m_1 + m_2/m_1**2)/c**2*dc_rdp_dot  +  ( ( (E_1_dot + E_2_dot)/(E_1*E_2) - (E_1 + E_2)/(E_1*E_2)**2*green)*dc_1dp + (E_1 + E_2)/(E_1*E_2)*dc_1dp_dot + (c_1_dot/(E_1*E_2) - c_1/(E_1*E_2)**2*green )* ( dE_1dp + dE_2dp - (E_1 + E_2)/(E_1*E_2)*purple)  +  c_1/(E_1*E_2)* ( dE_1dp_dot + dE_2dp_dot - ( (E_1_dot + E_2_dot)/(E_1*E_2) - (E_1 + E_2)/(E_1*E_2)**2*green )*purple - (E_1 + E_2)/(E_1*E_2)*yellow ) ) * (c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 + p**2)/c**2)  +  ( (E_1 + E_2)/(E_1*E_2)*dc_1dp + c_1/(E_1*E_2)* ( dE_1dp + dE_2dp - (E_1 + E_2)/(E_1*E_2)*purple ) ) * (c_1_dot/2*( 1/(E_1 + E_2)**2 - 1/(E_1*E_2) ) + c_1/2*(-2*(E_1_dot + E_2_dot)/(E_1 + E_2)**3 + green/(E_1*E_2)**2) - 4/c**2*(green/c**2 + 2*np.dot(p_vec, p_vec_dot) ) )  +  ( ( c_1_dot*(E_1 + E_2) + c_1*(E_1_dot + E_2_dot) )/(E_1*E_2) - c_1*(E_1 + E_2)/(E_1*E_2)**2*green )*( 1/2*dc_1dp*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) + c_1/2*( -2*(dE_1dp + dE_2dp)/(E_1 + E_2)**3 + purple/(E_1*E_2)**2 ) - 4/c**2*( purple/c**2 + 2*p_vec ) ) + c_1*(E_1 + E_2)/(E_1*E_2) * ( 1/2*dc_1dp_dot * (1/(E_1 + E_2)**2 - 1/(E_1*E_2)) + 1/2*dc_1dp*( -2*(E_1_dot + E_2_dot)/(E_1 + E_2)**3 + green/(E_1*E_2)**2 ) + c_1_dot/2*( -2*(dE_1dp + dE_2dp)/(E_1 + E_2)**3 + purple/(E_1*E_2)**2 )  +  c_1/2*( -2/(E_1 + E_2)**3*( dE_1dp_dot + dE_2dp_dot + purple*green  ) + 6*(dE_1dp + dE_2dp)/(E_1 + E_2)**4*(E_1_dot + E_2_dot) + yellow/(E_1*E_2)**2)  - 4/c**2*(yellow/c**2 + 2*p_vec_dot)  ) )

    a = a_kin + a_1PM + a_2PM

    return a

def boost(v_vec, a, dt):
    v_vec += a*dt
    return v_vec

def move(r_vec, v_vec, dt):
    r_vec += v_vec*dt
    return r_vec

### This runs the simulation

In [ ]:
def run_model(s, t_max, dt, DL = True, mode = '2PM', scattering = False):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    
    R_S = 2*(m_1 + m_2)*G/c**2

    dT = dt

    posr = np.zeros((2, int(t_max/dT)))
    pos1 = np.zeros((2, int(t_max/dT)))
    pos2 = np.zeros((2, int(t_max/dT)))
    pos_CM = np.zeros((2, int(t_max/dT)))

    r_init = rel_len(x_1, x_2)

    r_vec = x_1 - x_2
    R_vec = findrCM(x_1, x_2, m_1, m_2)
    v_vec = v_1 - v_2
    p_vec = v_1*m_1*γ(v_1, DL)

    
    for i in range(int(t_max/dT)):
        if rel_len(x_1, x_2) <= 5*R_S:
            print('Code interrupted: Radial Dip')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            break
        elif rel_len(x_1, x_2) >= 5*r_init:
            print('Code interrupted: Max distance')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            break
        elif np.linalg.norm(v_vec) > 1:
            print('Code interrupted: v > c')
            pos1, pos2, pos_CM = pos1[:,0:i], pos2[:,0:i], pos_CM[:,0:i]
            break
        

        posr[:,i] = r_vec
        pos_CM[:,i] = R_vec

        if mode == 'N':
            a = accelerationN(r_vec, v_vec, p_vec, m_1, m_2, DL)
        elif mode == 'EIH':
            a = accelerationEIH(r_vec, v_vec, p_vec, m_1, m_2, DL)
        elif mode == '1PM':
            a = acceleration1PM(r_vec, v_vec, p_vec, m_1, m_2, DL)
        elif mode == '2PM':
            a = acceleration2PM(r_vec, v_vec, p_vec, m_1, m_2, DL)
        else:
            print('Error: No mode selected')
            break

        if scattering == True:
            if np.linalg.norm(r_vec) <= 10*R_S:
                dT = 1e-6*np.linalg.norm(v_vec)/c/R_S
            # print(dT, r_vec)
        v_vec = boost(v_vec, a, dT)
        
        r_vec = move(r_vec, v_vec, dT)
        R_vec = findrCM(x_1, x_2, m_1, m_2)
        p_vec = γ(v_vec, DL)*v_vec*m_2/(m_1 + m_2)*m_1

    
    pos1 = pos_CM + m_2/(m_1 + m_2)*posr
    pos2 = pos_CM - m_1/(m_1 + m_2)*posr
    
    positions = np.array([pos1, pos2, pos_CM])
    return positions

### Plotting functions

In [ ]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def orbPlotter(positions, positionsN = 0, xlim = 0, ylim = 0, slice = slice(0, -1, 1), aspect = 1, filename='', CM = True, save = False, show = True, DL = True, N = False, figsize=(8,8)):
    if DL == False:
        x_1, x_2, x_cm = positions[0:3,:,slice]/149597871000
    else:
        x_1, x_2, x_cm = positions[0:3,:,slice]
    if CM == True:
        x_1 = x_1 - x_cm
        x_2 = x_2 - x_cm
        x_cm = np.zeros_like(x_1)

    fig, ax = plt.subplots(figsize=figsize)
    if type(xlim) != tuple:
        xlim = plotLimsTwoBody(x_1, x_2)[0]
    if type(ylim) != tuple:
        ylim = plotLimsTwoBody(x_1, x_2)[1]
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # if type(positionsN) == np.ndarray:
    #     if DL == False:
    #         x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]/149597871000
    #     else:
    #         x_1N, x_2N, x_cmN = positionsN[0:3,:,slice]
    #     if CM == True:
    #         x_1N = x_1N - x_cmN
    #         x_2N = x_2N - x_cmN
    #     ax.plot(x_1N[0], x_1N[1], 'y', lw=0.5, label='Newtonian motion')
    #     ax.plot(x_2N[0], x_2N[1], 'y', lw=0.5, label='Newtonian motion')

    
    ax.plot(x_cm[0], x_cm[1], 'g:',label = 'CM')
    ax.plot(x_cm[0][0], x_cm[1][0], 'gx',markersize=8)
    ax.plot(x_cm[0][-1], x_cm[1][-1], 'g.',markersize=8)

    ax.plot(x_1[0], x_1[1],'b')
    ax.plot(x_1[0][0], x_1[1][0], 'bx', label = 'm_1 start', markersize=15)
    ax.plot(x_1[0][-1], x_1[1][-1], 'b.', label = 'm_1 stop',markersize=15)

    ax.plot(x_2[0], x_2[1],'r')
    ax.plot(x_2[0][0], x_2[1][0], 'rx', label = 'm_2 start', markersize=15)
    ax.plot(x_2[0][-1], x_2[1][-1], 'r.', label = 'm_2 stop',markersize=15)

    # if DL == True:
    #     ax.set_xlabel('$R_S$', fontsize = 15)
    #     ax.set_ylabel('$R_S$', fontsize = 15)
    # else:
    #     ax.set_xlabel('$x \ [\mathrm{AU}]$', fontsize = 15)
    #     ax.set_ylabel('$y \ [\mathrm{AU}]$', fontsize = 15)  
    ax.grid(c='grey', alpha=0.2, ls ='--')
    ax.set_aspect(aspect)
    ax.set_title(f'{filename}', fontsize = 20)
    ax.legend(facecolor='grey', fontsize = 12, bbox_to_anchor=(1.01,1), loc='upper left')

    plt.title(f'{filename}', fontsize=25, y=1.08)
    fig.patch.set_facecolor('white')
    fig.tight_layout()
    if show == False:
        plt.close(fig)
    if save == True:
        fig.savefig(f'C:/Users/Productivity/Dropbox/_CARL/UNI/KURSER/ÅR 3/Bachelor Project/Git_2/Bachelor-Project/05 POST MINKOWSKIAN EXPANSION/Plots/{filename}.png', dpi=300, transparent = False)
    return

# Functions for extracting important observables

In [ ]:
def perihelionShift(pos):
    rs = []
    peakPositions = []
    peakAngles = []
    for i in range(len(pos[0])):
        rs.append(np.sqrt(pos[0,i]**2 + pos[1,i]**2))
    rs = np.array(rs)
    peakIndices = scipy.signal.argrelextrema(rs, np.greater)[0]
    for i in peakIndices:
        peakPositions.append(np.array([pos[0,i], pos[1,i]]))
    peakPositions = np.array(peakPositions)
    for i in range(len(peakPositions)-1):
            δΘ = np.arccos(np.dot(peakPositions[i], peakPositions[i+1])/(np.linalg.norm(peakPositions[i])*np.linalg.norm(peakPositions[i+1]) ))
            peakAngles.append(δΘ)
    perihelionShift = np.mean(peakAngles)
    return rs, peakPositions, peakAngles, perihelionShift

def scatteringAngle(pos):
    posi = pos[0,:,0] - np.array([0, pos[0,:,0][1]])
    posf = pos[0,:,-1] - np.array([0, pos[0,:,0][1]])
    return 180 - np.arccos(np.dot(posi, posf)/(np.linalg.norm(posi)*np.linalg.norm(posf)))*180/np.pi


def scatteringAngle_analytical(s, DL = True, mode = '2PM'):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s

    μ = m_1*m_2/(m_1 + m_2)

    r_vec = x_1 - x_2
    v_vec = v_1 - v_2
    # p_vec = γ(v_1, DL)*v_1*m_1
    p_vec = v_vec*μ*γ(v_vec, DL)

    p = np.linalg.norm(p_vec)
    v = np.linalg.norm(v_vec)
    r = np.linalg.norm(r_vec)

    b = r*np.sin(np.arccos(np.dot(r_vec, p_vec)/(r*p)))
    L = np.abs(np.cross(r_vec, p_vec))


    E_1 = np.sqrt((m_1*c**2)**2 + (p*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p*c)**2)
    c_1 = (m_1*m_2*c**2)**2 - 2*(E_1*E_2/c**2 + p**2)**2
    c_r = 3*m_1**2*((m_1*m_2)**2*c**4 - 5*(E_1*E_2/c**2 + p**2)**2)
    c_l = (m_2/m_1)**2*c_r
    p_0 = np.sqrt(((E_1*E_2/c**2 + p**2)**2 - (m_1*m_2*c**2)**2)/(E_1 + E_2)**2*c**2)
    f_1 = -2*c_1/(E_1/c + E_2/c)
    f_2 = -c/(2*(E_1 + E_2))*(c_r/m_1 + c_l/m_2)

    if mode == 'N':
        χ = 2*np.arctan(G*(m_1 + m_2)/(v**2*b))
    elif mode == '1PM':
        χ = G*f_1/(p_0*L*c**3)
    elif mode == '2PM':
        χ = G*f_1/(p_0*L*c**3) + G**2*f_2*np.pi/(2*L**2*c**5)
    else:
        χ = np.nan
        print('Error, no mode chosen')

    return np.array([χ*180/np.pi, L])

def otherScatteringAngleAnalytical(s, DL = True):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s

    r_vec = x_1 - x_2
    v_vec = v_1 - v_2
    p_vec = γ(v_1, DL)*v_1*m_1
    # p_vec = v_vec*μ*γ(v_vec, DL)

    μ = m_1*m_2/(m_1 + m_2)
    p = np.linalg.norm(p_vec)
    v = np.linalg.norm(v_vec)
    r = np.linalg.norm(r_vec)

    L = np.abs(np.cross(r_vec, p_vec))

    E_1 = np.sqrt((m_1*c**2)**2 + (p*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p*c)**2)
    c_1 = (m_1*m_2*c**2)**2 - 2*(E_1*E_2/c**2 + p**2)**2
    c_r = 3*m_1**2*((m_1*m_2)**2*c**4 - 5*(E_1*E_2/c**2 + p**2)**2)
    c_l = (m_2/m_1)**2*c_r
    p_0 = np.sqrt(((E_1*E_2/c**2 + p**2)**2 - (m_1*m_2*c**2)**2)/(E_1 + E_2))
    r_0 = L/p_0
    f_1 = -2*c_1/(E_1/c + E_2/c)
    f_2 = -1/(2*c*(E_1 + E_2))*(c_r/m_1 + c_l/m_2)

    r_plus = - G*f_1/(2*p_0**2) + np.sqrt((G*f_1)**2/(4*p_0**4) - G**2*f_2/p_0**2 + r_0**2)
    r_minus = - G*f_1/(2*p_0**2) - np.sqrt((G*f_1)**2/(4*p_0**4) - G**2*f_2/p_0**2 + r_0**2)

    χ = 4*r_0/np.sqrt(-r_minus*r_plus)*np.arccos(np.sqrt(r_plus/(r_plus - r_minus))) - np.pi
    return χ*180/np.pi

def TotalEnergy(s, DL = False, mode = '2PM'):
    if DL == True:
        G, c = 1, 1
    else:
        c = const.c
        G = const.G
    
    x_1, x_2, v_1, v_2, m_1, m_2 = s

    r_vec = x_1 - x_2
    v_vec = v_1 - v_2
    p_vec = γ(v_1, DL)*v_1*m_1

    p = np.linalg.norm(p_vec)
    v = np.linalg.norm(v_vec)
    r = np.linalg.norm(r_vec)

    E_1 = np.sqrt((m_1*c**2)**2 + (p*c)**2)
    E_2 = np.sqrt((m_2*c**2)**2 + (p*c)**2)
    c_1 = (m_1*m_2*c**2)**2 - 2*(E_1*E_2/c**2 + p**2)**2
    c_r = 3*m_1**2*((m_1*m_2)**2*c**4 - 5*(E_1*E_2/c**2 + p**2)**2)
    c_l = (m_2/m_1)**2*c_r

    if mode == 'N':
        E = (1/m_1 + 1/m_2)*p**2 - G*m_1*m_2/r
    elif mode == 'EIH':
        E = 1/2*(1/m_1 + 1/m_2)*p**2 - 1/8*(1/m_1**3 + 1/m_2**3)*p**4/c**2 + G**2*m_1*m_2*(m_1 + m_2)/(2*r**2*c**2) - G/r*( m_1*m_2 + (1/2 + 3/2*(m_1 + m_2)**2/(m_1*m_2))*p**2/c**2 + 1/2*np.dot(p_vec, r_vec)**2/(r*c)**2 )
    elif mode == '1PM':
        E = E_1 + E_2 + 1/(E_1*E_2)*G*c_1/r
    elif mode == '2PM':
        E = E_1 + E_2 + 1/(E_1*E_2)*G*c_1/r + G**2/(E_1*E_2*r**2)*( 1/4*(c_r/m_1 + c_l/m_2)/c**2 + c_1*(E_1 + E_2)/(E_1*E_2)*( c_1/2*(1/(E_1 + E_2)**2 - 1/(E_1*E_2)) - 4*(E_1*E_2/c**2 + p**2) ) )
    else:
        print('Error: mode not chosen')
    
    return E, E - (m_1 + m_2)*c**2
